In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from tpot import TPOTClassifier

In [21]:
df = pd.read_csv("tested.csv")

In [23]:
df = df.drop(["PassengerId", "Ticket"], axis=1)

In [24]:
df["Title"] = df["Name"].apply(lambda x: x.split(",")[1].strip().split(" ")[0])

In [26]:
le = LabelEncoder().fit(df["Sex"])
le_nm = dict(zip(le.classes_, le.transform(le.classes_)))
df["Sex"] = df["Sex"].apply(lambda x: le_nm[x])

In [28]:
df["Title"].value_counts()

Mr.        240
Miss.       78
Mrs.        72
Master.     21
Col.         2
Rev.         2
Ms.          1
Dr.          1
Dona.        1
Name: Title, dtype: int64

In [29]:
common_titles = ["Mr.", "Miss.", "Mrs."]

In [30]:
df["Title"] = [0 if x in common_titles else 1 for x in df["Title"]]

In [31]:
df["Cabin"] = [0 if str(x) == "nan" else 1 for x in df["Cabin"]]

In [33]:
embark_df = pd.get_dummies(df["Embarked"])

In [35]:
df = pd.concat([df, embark_df], axis=1)

In [37]:
df = df.drop(["Name", "Embarked"], axis=1)

In [40]:
X = df.drop(["Survived"], axis=1)
y = df["Survived"]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4242)

In [42]:
pipe = TPOTClassifier(scoring="f1", max_time_mins=5, random_state=4242, verbosity=2)
pipe.fit(X_train, y_train)

Imputing missing values in feature set


Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Generation 3 - Current best internal CV score: 1.0

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

5.07 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=1, max_features=0.9000000000000001, min_samples_leaf=20, min_samples_split=10, n_estimators=100, subsample=0.45)


TPOTClassifier(max_time_mins=5, random_state=4242, scoring='f1', verbosity=2)

In [43]:
y_pred = pipe.predict(X_test)

Imputing missing values in feature set


In [45]:
accuracy_score(y_test, y_pred)

1.0

In [46]:
f1_score(y_test, y_pred)

1.0

In [47]:
pipe.export("titanic.py")